In [106]:
#Workshop 1 : โหลดข้อมูล
# โหลดข้อมูล ตรวจดูข้อมูล และเตรียมข้อมมูลเบื้องต้น
from sklearn.datasets import load_breast_cancer
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

cancer = load_breast_cancer()
cancer.keys() # ดู key ในชุดข้อมูลมีอะไรบ้าง

# ต้องการใช้ feature_names เก็บไว้ DataFrame
df = pd.DataFrame(cancer.data, columns=cancer.feature_names) # ข้อมูล และ ชื่อตัวแปร
df['class'] = cancer.target # Class / Target / Label (เป็นหรือไม่เป็นมะเร็ง)
df.head() # ดู head

df.isnull().sum() # ตรวจดู missing data



mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
class                      0
dtype: int64

In [107]:
#พล็อตดู 2 กราฟคู่กันดู
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8,3))
ax1.set_title('mean radius')
ax1.boxplot(df['mean radius'])

ax2.set_title('mean texture')
ax2.boxplot(df['mean texture'])
plt.show()

In [108]:
# การวิเคราะห์มะเร็ง ทรวงอกบทนี้จะใช้เพียง 2 ตัวแปร  mean radius and mean texture
# โดยตัด outlier ที่ 25 และ 38 ตามลำดับ
df = df[df['mean radius']<25]
df = df[df['mean texture']<38]
df.shape

(563, 31)

In [109]:
# ตรวจตูข้อมูลโดยการพล็อต Scatter (ทำ Data Visualization)
sns.set_style('whitegrid')  # สีพื้น
plt.figure(figsize=(5.3,3.2)) # ขนาดกราฟ

sns.scatterplot('mean radius','mean texture',data=df, hue='class', style='class',
               alpha=0.9, edgecolor='w', s=80)
plt.grid(False) # ซ่อน grid
plt.legend(loc='upper left',bbox_to_anchor=(1,1))
plt.show()


In [75]:
# Workshop 2 : Case 1 ROC(Reciver Operating Characteristic curve) AUC(Area Under the Curve) ของ Modelที่ดีที่สุด
# ทำนาย 100 เคสถูก 100 เคส
# สร้างข้อมูลจากชุดข้อมูล โดยตัดส่วนที่ข้อมูลซ้อนทับกันใช้ดป็นกรณีศึกษา
df['mean ra'] = np.where(df['class']==0, df['mean radius']+7.5, df['mean radius']) # เลือก row == 0 , จากค่า mean radius +7.5
df['mean ra'].head(3)

0    25.49
1    28.07
2    27.19
Name: mean ra, dtype: float64

In [76]:
# ตรวจตูข้อมูลโดยการพล็อต Scatter (ทำ Data Visualization)
# ข้อมูลที่ทำการคัดเอาแต่ส่วนที่แยกกัน
plt.figure(figsize=(5.3, 3.2))
plt.rcParams['axes.edgecolor'] = '150'
plt.rcParams['axes.linewidth'] = 1.25
sns.scatterplot('mean ra','mean texture', data =df, hue='class', style='class',
               alpha=0.9, edgecolor='w', s=80)
plt.grid(False)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [77]:
# ตรวจดู Histrogram(การกระจายของข้อมูล) by displot

plt.figure(figsize=(4.5, 3.2))
sns.distplot(df['mean ra'][df['class']==0], label='0', color='b')
sns.distplot(df['mean ra'][df['class']==1], label='1', color='r')
plt.legend()
plt.grid(False)
plt.show()

In [78]:
# เตรียมข้อมูลเพื่อไป train model X(Feature = mean ra, mean texture) y(class)
columns = ['mean ra','mean texture']
X = df[columns]
y = df['class']
X.shape , y.shape , y.unique() # ดู จำนวน X y และ ข้อมูล y มีรูปแบบใดบ้าง
X[:3]

,mean ra,mean texture
0,25.49,10.38
1,28.07,17.77
2,27.19,21.25


In [79]:
#Train Test
# แบ่งข้อมูล -> สร้าง model LogisticRegression -> Test
#1 split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=10)

In [80]:
#2 model building
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [81]:
#3 การประเมิณ model
from sklearn.metrics import classification_report, confusion_matrix

#Accuracy
print("Score {:.3f}".format(model.score(X_test, y_test)))

#Precision
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No','Yes']))
cm = confusion_matrix(y_test, y_pred)
print(cm)
# ใช้งาน scikitplot ทำ HeatMap  Condusion matrix
import scikitplot as skplot
sns.set(rc={'figure.figsize':(2.2,2)})
skplot.metrics.plot_confusion_matrix(y_test, y_pred)
plt.show()

#ทำ Cross validation (k-fold k=10)
from sklearn.model_selection import cross_val_score
model = LogisticRegression(solver='lbfgs')
cvs = cross_val_score(model, X, y, cv=10)
print('cross val score {}'.format(cvs.round(2)))
print('Average(%) = {:.2f}'.format(cvs.mean()*100))

Score 1.000
              precision    recall  f1-score   support

          No       1.00      1.00      1.00        63
         Yes       1.00      1.00      1.00       106

    accuracy                           1.00       169
   macro avg       1.00      1.00      1.00       169
weighted avg       1.00      1.00      1.00       169

[[ 63   0]
 [  0 106]]
cross val score [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Average(%) = 100.00


In [82]:
#ทำ ROC curve
#ต้องใช้ model.predict_proba()
# ถ้าใช้ model decision tree ต้องกำหนด max_depth ด้วย เนื่องจาก ความน่าจะเป็นอาจออกมาแค่ 0 , 1 
model.fit(X_train, y_train) # Train อีกครั้ง

y_pred_prob = model.predict_proba(X_test) # Predict ด้วย  .predict_proba
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob[:,1]) # ค่าที่ใช้ พล็อต ROC
y_pred_prob[:10].round(3)
'''นำค่าy(จริง) กับ y_pred_prob เก็บใส่ DataFrame'''
df_result = pd.DataFrame(y_test)
df_result['y_pred'] = y_pred
df_result['y_pred_prob0'] = y_pred_prob[:,0] # ความน่าจะเป็น ของผลทำนาย Negative
df_result['y_pred_pred1'] = y_pred_prob[:,1] # ความน่าจะเป็น ของผลทำนาย Positive
df_result.head().round(3)
#พล็อต ROC curve

from sklearn.metrics import roc_curve, auc
#%matplotlib
ll = [0,1] #Lower Left
ur = [0,1] #Upper Right

plt.figure(figsize=(4,4))
y_pred_prob = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
plt.plot(fpr,tpr)
plt.plot(ll,ur,'--r') # --r = color red ลาก -- จาก ll -> ur
plt.title('ROC Curve')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.grid(False)
plt.show()

auc = auc(fpr,tpr)
auc

1.0

In [83]:
# Workshop 3 : Case  2 ROC AUC Model แม่นยำ 88%
# ให้ใช้ 'mean ra' = 'mean radius'
df['mean ra'] = df['mean radius'] #ค่าเดิม ข้อมูลต้นฉบับ

# ตรวจตูข้อมูลโดยการพล็อต Scatter (ทำ Data Visualization)
# ข้อมูลที่ทำการคัดเอาแต่ส่วนที่แยกกัน
plt.figure(figsize=(5.3, 3.2))
plt.rcParams['axes.edgecolor'] = '150'
plt.rcParams['axes.linewidth'] = 1.25
sns.scatterplot('mean ra','mean texture', data =df, hue='class', style='class',
               alpha=0.9, edgecolor='w', s=80)
plt.grid(False)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [84]:
# ตรวจดู Histrogram(การกระจายของข้อมูล) by displot

plt.figure(figsize=(4.5, 3.2))
sns.distplot(df['mean ra'][df['class']==0], label='0', color='b')
sns.distplot(df['mean ra'][df['class']==1], label='1', color='r')
plt.legend()
plt.grid(False)
plt.show()

In [85]:
# เตรียมข้อมูลเพื่อไป train model X(Feature = mean ra, mean texture) y(class)
columns = ['mean ra','mean texture']
X = df[columns]
y = df['class']
X.shape , y.shape , y.unique() # ดู จำนวน X y และ ข้อมูล y มีรูปแบบใดบ้าง
X[:3]

,mean ra,mean texture
0,17.99,10.38
1,20.57,17.77
2,19.69,21.25


In [86]:
#Train Test
# แบ่งข้อมูล -> สร้าง model LogisticRegression -> Test
#1 split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=10)

In [87]:
#2 model building
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [88]:
#3 การประเมิณ model
from sklearn.metrics import classification_report, confusion_matrix

#Accuracy
print("Score {:.3f}".format(model.score(X_test, y_test)))

#Precision
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No','Yes']))
cm = confusion_matrix(y_test, y_pred)
print(cm)
# ใช้งาน scikitplot ทำ HeatMap  Condusion matrix
import scikitplot as skplot
sns.set(rc={'figure.figsize':(2.2,2)})
skplot.metrics.plot_confusion_matrix(y_test, y_pred)
plt.show()

#ทำ Cross validation (k-fold k=10)
from sklearn.model_selection import cross_val_score
model = LogisticRegression(solver='lbfgs')
cvs = cross_val_score(model, X, y, cv=10)
print('cross val score {}'.format(cvs.round(2)))
print('Average(%) = {:.2f}'.format(cvs.mean()*100))

Score 0.911
              precision    recall  f1-score   support

          No       0.94      0.81      0.87        63
         Yes       0.90      0.97      0.93       106

    accuracy                           0.91       169
   macro avg       0.92      0.89      0.90       169
weighted avg       0.91      0.91      0.91       169

[[ 51  12]
 [  3 103]]
cross val score [0.84 0.86 0.89 0.86 0.88 0.88 0.95 0.93 0.93 0.86]
Average(%) = 88.64


In [89]:
#ทำ ROC curve
#ต้องใช้ model.predict_proba()
# ถ้าใช้ model decision tree ต้องกำหนด max_depth ด้วย เนื่องจาก ความน่าจะเป็นอาจออกมาแค่ 0 , 1 
model.fit(X_train, y_train) # Train อีกครั้ง

y_pred_prob = model.predict_proba(X_test) # Predict ด้วย  .predict_proba
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob[:,1]) # ค่าที่ใช้ พล็อต ROC
y_pred_prob[:10].round(3)
'''นำค่าy(จริง) กับ y_pred_prob เก็บใส่ DataFrame'''
df_result = pd.DataFrame(y_test)
df_result['y_pred'] = y_pred
df_result['y_pred_prob0'] = y_pred_prob[:,0] # ความน่าจะเป็น ของผลทำนาย Negative
df_result['y_pred_pred1'] = y_pred_prob[:,1] # ความน่าจะเป็น ของผลทำนาย Positive
df_result.head().round(3)
#พล็อต ROC curve

from sklearn.metrics import roc_curve, auc
#%matplotlib
ll = [0,1] #Lower Left
ur = [0,1] #Upper Right

plt.figure(figsize=(4,4))
y_pred_prob = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
plt.plot(fpr,tpr)
plt.plot(ll,ur,'--r') # --r = color red ลาก -- จาก ll -> ur
plt.title('ROC Curve')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.grid(False)
plt.show()

auc = auc(fpr,tpr)
auc

0.9634621144055106

In [90]:
# Workshop 4 : Case  2 ROC AUC Model แม่นยำ 70%
# สร้างข้อมูลจากชุดข้อมูล โดยใช้ส่วนที่ข้อมูลซ้อนทับกันบางส่วนใช้เป็นกรณีศึกษา
df['mean ra'] = np.where(df['class']==0, df['mean radius']-5, df['mean radius']) # เลือก row == 0 , จากค่า mean radius -5
df['mean ra'].head(3)

# ตรวจตูข้อมูลโดยการพล็อต Scatter (ทำ Data Visualization)
# ข้อมูลที่ทำการคัดเอาแต่ส่วนที่แยกกัน
plt.figure(figsize=(5.3, 3.2))
plt.rcParams['axes.edgecolor'] = '150'
plt.rcParams['axes.linewidth'] = 1.25
sns.scatterplot('mean ra','mean texture', data =df, hue='class', style='class',
               alpha=0.9, edgecolor='w', s=80)
plt.grid(False)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [91]:
# ตรวจดู Histrogram(การกระจายของข้อมูล) by displot

plt.figure(figsize=(4.5, 3.2))
sns.distplot(df['mean ra'][df['class']==0], label='0', color='b')
sns.distplot(df['mean ra'][df['class']==1], label='1', color='r')
plt.legend()
plt.grid(False)
plt.show()

In [92]:
# เตรียมข้อมูลเพื่อไป train model X(Feature = mean ra, mean texture) y(class)
columns = ['mean ra','mean texture']
X = df[columns]
y = df['class']
X.shape , y.shape , y.unique() # ดู จำนวน X y และ ข้อมูล y มีรูปแบบใดบ้าง
X[:3]

,mean ra,mean texture
0,12.99,10.38
1,15.57,17.77
2,14.69,21.25


In [93]:
#Train Test
# แบ่งข้อมูล -> สร้าง model LogisticRegression -> Test
#1 split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=10)

In [94]:
#2 model building
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
#3 การประเมิณ model
from sklearn.metrics import classification_report, confusion_matrix

#Accuracy
print("Score {:.3f}".format(model.score(X_test, y_test)))

#Precision
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No','Yes']))
cm = confusion_matrix(y_test, y_pred)
print(cm)
# ใช้งาน scikitplot ทำ HeatMap  Condusion matrix
import scikitplot as skplot
sns.set(rc={'figure.figsize':(2.2,2)})
skplot.metrics.plot_confusion_matrix(y_test, y_pred)
plt.show()

#ทำ Cross validation (k-fold k=10)
from sklearn.model_selection import cross_val_score
model = LogisticRegression(solver='lbfgs')
cvs = cross_val_score(model, X, y, cv=10)
print('cross val score {}'.format(cvs.round(2)))
print('Average(%) = {:.2f}'.format(cvs.mean()*100))

Score 0.716
              precision    recall  f1-score   support

          No       0.69      0.43      0.53        63
         Yes       0.72      0.89      0.80       106

    accuracy                           0.72       169
   macro avg       0.71      0.66      0.66       169
weighted avg       0.71      0.72      0.70       169

[[27 36]
 [12 94]]
cross val score [0.74 0.74 0.72 0.61 0.77 0.77 0.79 0.61 0.77 0.46]
Average(%) = 69.61


In [96]:
#ทำ ROC curve
#ต้องใช้ model.predict_proba()
# ถ้าใช้ model decision tree ต้องกำหนด max_depth ด้วย เนื่องจาก ความน่าจะเป็นอาจออกมาแค่ 0 , 1 
model.fit(X_train, y_train) # Train อีกครั้ง

y_pred_prob = model.predict_proba(X_test) # Predict ด้วย  .predict_proba
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob[:,1]) # ค่าที่ใช้ พล็อต ROC
y_pred_prob[:10].round(3)
'''นำค่าy(จริง) กับ y_pred_prob เก็บใส่ DataFrame'''
df_result = pd.DataFrame(y_test)
df_result['y_pred'] = y_pred
df_result['y_pred_prob0'] = y_pred_prob[:,0] # ความน่าจะเป็น ของผลทำนาย Negative
df_result['y_pred_pred1'] = y_pred_prob[:,1] # ความน่าจะเป็น ของผลทำนาย Positive
df_result.head().round(3)
#พล็อต ROC curve

from sklearn.metrics import roc_curve, auc
#%matplotlib
ll = [0,1] #Lower Left
ur = [0,1] #Upper Right

plt.figure(figsize=(4,4))
y_pred_prob = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
plt.plot(fpr,tpr)
plt.plot(ll,ur,'--r') # --r = color red ลาก -- จาก ll -> ur
plt.title('ROC Curve')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.grid(False)
plt.show()

auc = auc(fpr,tpr)
auc

0.8129679544773883

In [97]:
# Workshop 5 : Case  2 ROC AUC Model แม่นยำ 50% (แย่ที่สุด)
# สร้างข้อมูลจากชุดข้อมูล โดยใช้ส่วนที่ข้อมูลซ้อนทับกันทุกส่วนใช้เป็นกรณีศึกษา

#ตัดข้อมูลบางส่วนออก
idx = df[df['class'] == 1].index
df.drop(idx, axis=0, inplace=True)
print(df.shape)
df2 = df.copy()
df2['class'] = 1
df2.head()
df = df.append(df2, ignore_index=True)
df['mean ra'] = np.where(df['class']==0, df['mean radius']+1, df['mean radius']) # เลือก row == 0 , จากค่า mean radius +1 

# ตรวจตูข้อมูลโดยการพล็อต Scatter (ทำ Data Visualization)
# ข้อมูลที่ทำการคัดเอาแต่ส่วนที่แยกกัน
plt.figure(figsize=(5.3, 3.2))
plt.rcParams['axes.edgecolor'] = '150'
plt.rcParams['axes.linewidth'] = 1.25
sns.scatterplot('mean ra','mean texture', data =df, hue='class', style='class',
               alpha=0.9, edgecolor='w', s=80)
plt.grid(False)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.show()

(206, 32)


In [98]:
# ตรวจดู Histrogram(การกระจายของข้อมูล) by displot

plt.figure(figsize=(4.5, 3.2))
sns.distplot(df['mean ra'][df['class']==0], label='0', color='b')
sns.distplot(df['mean ra'][df['class']==1], label='1', color='r')
plt.legend()
plt.grid(False)
plt.show()

In [99]:
# เตรียมข้อมูลเพื่อไป train model X(Feature = mean ra, mean texture) y(class)
columns = ['mean ra','mean texture']
X = df[columns]
y = df['class']
X.shape , y.shape , y.unique() # ดู จำนวน X y และ ข้อมูล y มีรูปแบบใดบ้าง
X[:3]

,mean ra,mean texture
0,18.99,10.38
1,21.57,17.77
2,20.69,21.25


In [100]:
#Train Test
# แบ่งข้อมูล -> สร้าง model LogisticRegression -> Test
#1 split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=10)

In [101]:
#2 model building
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [116]:
#3 การประเมิณ model
from sklearn.metrics import classification_report, confusion_matrix

#Accuracy
print("Score {:.3f}".format(model.score(X_test, y_test)))

#Precision
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No','Yes']))
cm = confusion_matrix(y_test, y_pred)
print(cm)
# ใช้งาน scikitplot ทำ HeatMap  Condusion matrix
import scikitplot as skplot
sns.set(rc={'figure.figsize':(2.2,2)})
skplot.metrics.plot_confusion_matrix(y_test, y_pred)
plt.show()

#ทำ Cross validation (k-fold k=10)
from sklearn.model_selection import cross_val_score
model = LogisticRegression(solver='lbfgs')
cvs = cross_val_score(model, X, y, cv=10)
print('cross val score {}'.format(cvs.round(2)))
print('Average(%) = {:.2f}'.format(cvs.mean()*100))

Score 0.911
              precision    recall  f1-score   support

          No       0.94      0.81      0.87        63
         Yes       0.90      0.97      0.93       106

    accuracy                           0.91       169
   macro avg       0.92      0.89      0.90       169
weighted avg       0.91      0.91      0.91       169

[[ 51  12]
 [  3 103]]
cross val score [0.84 0.86 0.89 0.86 0.88 0.88 0.95 0.93 0.93 0.86]
Average(%) = 88.64


In [103]:
#ทำ ROC curve
#ต้องใช้ model.predict_proba()
# ถ้าใช้ model decision tree ต้องกำหนด max_depth ด้วย เนื่องจาก ความน่าจะเป็นอาจออกมาแค่ 0 , 1 
model.fit(X_train, y_train) # Train อีกครั้ง

y_pred_prob = model.predict_proba(X_test) # Predict ด้วย  .predict_proba
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob[:,1]) # ค่าที่ใช้ พล็อต ROC
y_pred_prob[:10].round(3)
'''นำค่าy(จริง) กับ y_pred_prob เก็บใส่ DataFrame'''
df_result = pd.DataFrame(y_test)
df_result['y_pred'] = y_pred
df_result['y_pred_prob0'] = y_pred_prob[:,0] # ความน่าจะเป็น ของผลทำนาย Negative
df_result['y_pred_pred1'] = y_pred_prob[:,1] # ความน่าจะเป็น ของผลทำนาย Positive
df_result.head().round(3)
#พล็อต ROC curve

from sklearn.metrics import roc_curve, auc
#%matplotlib
ll = [0,1] #Lower Left
ur = [0,1] #Upper Right

plt.figure(figsize=(4,4))
y_pred_prob = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])
plt.plot(fpr,tpr)
plt.plot(ll,ur,'--r') # --r = color red ลาก -- จาก ll -> ur
plt.title('ROC Curve')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.grid(False)
plt.show()

auc = auc(fpr,tpr)
auc

0.5022164276401564

In [117]:
#Workshop 6: กำหนดค่า Threshpld สำหรับ Model
df['mean ra'] = df['mean radius']

In [118]:
plt.figure(figsize=(4.7, 3.2))
sns.scatterplot('mean ra','mean texture', data=df, hue='class', style='class',
               alpha=0.9, edgecolor='w', s=80)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [119]:
columns = ['mean ra','mean texture']
X = df[columns]
y = df['class']
X.shape, y.shape, y.unique()

((563, 2), (563,), array([0, 1]))

In [120]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [121]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

# y_pred = model(X_train, y_train) # default threshold = .5 50-50 (ค่าอยู่ได้0-1)
#กำหนด threshold
threshold = .34
y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = y_pred_prob > threshold
print(classification_report(y_test, y_pred, target_names=['No','Yes']))
cm = confusion_matrix(y_test, y_pred)
print(cm)
# ใช้งาน scikitplot ทำ HeatMap  Condusion matrix
import scikitplot as skplot
sns.set(rc={'figure.figsize':(2.2,2)})
skplot.metrics.plot_confusion_matrix(y_test, y_pred)
plt.show()


              precision    recall  f1-score   support

          No       1.00      0.76      0.86        63
         Yes       0.88      1.00      0.93       106

    accuracy                           0.91       169
   macro avg       0.94      0.88      0.90       169
weighted avg       0.92      0.91      0.91       169

[[ 48  15]
 [  0 106]]


In [123]:
threshold = .5
y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = y_pred_prob > threshold
print(classification_report(y_test, y_pred, target_names=['No','Yes']))
cm = confusion_matrix(y_test, y_pred)
print(cm)
# ใช้งาน scikitplot ทำ HeatMap  Condusion matrix
import scikitplot as skplot
sns.set(rc={'figure.figsize':(2.2,2)})
skplot.metrics.plot_confusion_matrix(y_test, y_pred)
plt.show()

              precision    recall  f1-score   support

          No       0.94      0.81      0.87        63
         Yes       0.90      0.97      0.93       106

    accuracy                           0.91       169
   macro avg       0.92      0.89      0.90       169
weighted avg       0.91      0.91      0.91       169

[[ 51  12]
 [  3 103]]


In [124]:
threshold = .98 # มีความแข็งมากส่งผลให้การคัดการเป็นมะเร็งต้องมีความแม่นยำ 100%
y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = y_pred_prob > threshold
print(classification_report(y_test, y_pred, target_names=['No','Yes']))
cm = confusion_matrix(y_test, y_pred)
print(cm)
# ใช้งาน scikitplot ทำ HeatMap  Condusion matrix
import scikitplot as skplot
sns.set(rc={'figure.figsize':(2.2,2)})
skplot.metrics.plot_confusion_matrix(y_test, y_pred)
plt.show()

              precision    recall  f1-score   support

          No       0.48      1.00      0.65        63
         Yes       1.00      0.35      0.52       106

    accuracy                           0.59       169
   macro avg       0.74      0.67      0.58       169
weighted avg       0.81      0.59      0.57       169

[[63  0]
 [69 37]]


ค่า Threshold ต่ำ = ความไวสูง (TPR สูง) ตรวจจับให้ผล Positive ง่ายขึ้น (High sensitivity)
แต่ทำให้ความผิดพลาดได้เพิ่มขึ้น(FPR สูงตาม) 